In [2]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
from scipy.special import erf
from matplotlib.colors import LogNorm, Normalize
from matplotlib.ticker import StrMethodFormatter, NullFormatter
import matplotlib.patheffects as path_effects
import scipy.ndimage
import illustris_python as il

# Universal constants
c = 2.99792458e10          # Speed of light [cm/s]
kB = 1.380648813e-16       # Boltzmann's constant [g cm^2/s^2/K]
h = 6.626069573e-27        # Planck's constant [erg/s]
mH = 1.6735327e-24         # Mass of hydrogen atom [g]
me = 9.109382917e-28       # Electron mass [g]
ee = 4.80320451e-10        # Electron charge [g^(1/2) cm^(3/2) / s]

# Emperical unit definitions
Msun = 1.988435e33         # Solar mass [g]
Lsun = 3.839e33            # Solar luminosity [erg/s]
Zsun = 0.0134              # Solar metallicity (mass fraction)
arcsec = 648000. / np.pi   # arseconds per radian
pc = 3.085677581467192e18  # Units: 1 pc  = 3e18 cm
kpc = 1e3 * pc             # Units: 1 kpc = 3e21 cm
Mpc = 1e6 * pc             # Units: 1 Mpc = 3e24 cm
km = 1e5                   # Units: 1 km  = 1e5  cm
angstrom = 1e-8            # Units: 1 angstrom = 1e-8 cm
day = 86400.               # Units: 1 day = 24 * 3600 seconds
yr = 365.24 * day          # Units: 1 year = 365.24 days
kyr = 1e3 * yr             # Units: 1 Myr = 10^6 yr
Myr = 1e6 * yr             # Units: 1 Myr = 10^6 yr
lambda_1216 = 1215.67 * angstrom # Lyman-alpha wavelength [cm]
lambda_1500 = 1500. * angstrom # Continuum wavelength [cm]
lambda_2500 = 2500. * angstrom # Continuum wavelength [cm]
R_10pc = 10. * pc              # Reference distance for continuum [cm]
fnu_1216_fac = lambda_1216**2 / (4. * np.pi * c * R_10pc**2 * angstrom)
fnu_1500_fac = lambda_1500**2 / (4. * np.pi * c * R_10pc**2 * angstrom)
fnu_2500_fac = lambda_2500**2 / (4. * np.pi * c * R_10pc**2 * angstrom)
E_AGN = 5.29e-11 # Mean photon energy [erg]
E_Lya = h * c / lambda_1216 # Lyman-alpha energy [erg]

In [1]:
def read_Lya(snap, sim):
    print(f'Reading snapshot {snap} ...')
    basePath = f'/nfs/mvogelsblab001/Thesan/{sim}/output'
    fields = ['SubhaloMass','SubhaloSFRinRad', 'SubhaloLenType', 'SubhaloMassType', 'SubhaloVelDisp', 'SubhaloPos', 'SubhaloGrNr']
#     fields = ['SubhaloLenType', 'SubhaloMass', 'SubhaloMassInRadType', 'SubhaloStellarPhotometrics', 'SubhaloStarMetallicity']
#   'SubhaloSFRinRad', 'SubhaloCM', 'SubhaloPos', 'SubhaloVel', 'SubhaloVelDisp']
    print('test0')
    s = il.groupcat.loadSubhalos(basePath, snap, fields=fields)

    print('test')
    s['snap'] = snap
    s['basePath'] = basePath
    Lya_filename = basePath + f'/../postprocessing/Lya/Lya_{snap:03d}.hdf5'
    with h5py.File(Lya_filename, 'r') as f:
        g = f['Header']
        for key in ['BoxSize', 'EscapeFraction', 'Omega0', 'OmegaBaryon', 'OmegaLambda', 'HubbleParam', 'Time', 'Redshift']:
            s[key] = g.attrs[key]
#         BoxSize = g.attrs['BoxSize']
#         EscapeFraction = g.attrs['EscapeFraction']
#         Omega0 = g.attrs['Omega0']
#         OmegaBaryon = g.attrs['OmegaBaryon']
#         OmegaLambda = g.attrs['OmegaLambda']
        h = g.attrs['HubbleParam']
        a = g.attrs['Time']
        z = g.attrs['Redshift']
        UnitLength_in_cm = g.attrs['UnitLength_in_cm']
        UnitMass_in_g = g.attrs['UnitMass_in_g']
        UnitVelocity_in_cm_per_s = g.attrs['UnitVelocity_in_cm_per_s']
        UnitTime_in_s = UnitLength_in_cm / UnitVelocity_in_cm_per_s
        UnitEnergy_in_cgs = UnitMass_in_g * UnitVelocity_in_cm_per_s * UnitVelocity_in_cm_per_s
        UnitLum_in_cgs = UnitEnergy_in_cgs / UnitTime_in_s
        length_to_cgs = a * UnitLength_in_cm / h
        length_to_kpc = length_to_cgs / kpc
        volume_to_cgs = length_to_cgs * length_to_cgs * length_to_cgs
        s['BoxSize_Mpc'] = s['BoxSize'] * length_to_cgs / Mpc # Box size in Mpc (physical)
        s['V_box_Mpc3'] = s['BoxSize_Mpc']**3 # Box volume in Mpc^3 (physical)
        s['V_box_cMpc3'] = s['V_box_Mpc3'] / s['Time']**3
        mass_to_cgs = UnitMass_in_g / h
        mass_to_Msun = mass_to_cgs / Msun
#         mask = (s['SubhaloLenType'][:,1] >= 0) # No masking
#         mask = ((s['SubhaloMassInRadType'][:,4] > 0. ) &
#                 (s['SubhaloLenType'][:,4] >= 4) &
#                 (s['SubhaloLenType'][:,1] >= 32)) # Minimally resolved

#         s['SubhaloMass'] = mass_to_Msun * s['SubhaloMass'] # Msun
        s['SubhaloMass'] = s['SubhaloMass'] * 1e10 / h # Msun
    
#         s['SubhaloMassInRadType'] = mass_to_Msun * s['SubhaloMassInRadType'] # Msun
#         s['SubhaloMassStars'] = mass_to_Msun * s['SubhaloMassInRadType'][:,4] # Msun
#         s['M1450'] = s['SubhaloStellarPhotometrics'][:,0]
        s['Lya'] = UnitLum_in_cgs * f['Subhalo']['LyaLum'][:].astype(np.float64) # Lya = LyaCol + LyaRec + LyaStars
        s['LyaCol'] = UnitLum_in_cgs * f['Subhalo']['LyaLumCol'][:].astype(np.float64) # erg/s
        s['LyaRec'] = UnitLum_in_cgs * f['Subhalo']['LyaLumRec'][:].astype(np.float64) # erg/s
        s['LyaStars'] = UnitLum_in_cgs * f['Subhalo']['LyaLumStars'][:].astype(np.float64) # erg/s
        s['L1216'] = UnitLum_in_cgs * f['Subhalo']['1216LumStars'][:].astype(np.float64) # erg/s/Angstrom
        s['L1500'] = UnitLum_in_cgs * f['Subhalo']['1500LumStars'][:].astype(np.float64) # erg/s/Angstrom
        s['L2500'] = UnitLum_in_cgs * f['Subhalo']['2500LumStars'][:].astype(np.float64) # erg/s/Angstrom
#         s['IonAGN'] = UnitLum_in_cgs * f['Subhalo']['IonLumAGN'][:].astype(np.float64) # erg/s
#         mask = (s['IonAGN'] > 0.)
#         y_AGN = s['IonAGN'][mask] / E_AGN # AGN ionizing photon rate [photons/s]
#         y_stars = s['LyaStars'][mask] / (0.68 * E_Lya * (1. - s['EscapeFraction'])) # Star ionizing photon rate [photons/s]
#         s['f_AGN'] = y_AGN / (y_AGN + y_stars)
#         for key in ['SubhaloMass', 'SubhaloMassStars', 'Lya', 'LyaCol', 'LyaRec', 'LyaStars', 'L1216', 'L1500', 'L2500', 'IonAGN']:
#             if key == 'L1500':
#                 s['SubhaloStarMetallicity'] = s['SubhaloStarMetallicity'][s[key]>0.] / 0.0127 # Solar units
#             s[key] = s[key][s[key]>0.]
#         s['M1450'] = s['M1450'][s['M1450']<0.]
        s['M1216'] = -2.5 * np.log10(fnu_1216_fac * s['L1216']) - 48.6 # Continuum absolute magnitude
        s['M1500'] = -2.5 * np.log10(fnu_1500_fac * s['L1500']) - 48.6 # Continuum absolute magnitude
        s['M2500'] = -2.5 * np.log10(fnu_2500_fac * s['L2500']) - 48.6 # Continuum absolute magnitude
        # Note: z = [6, 7, 8, 9, 10]  =>  dtm = [0.11, 0.08, 0.06, 0.0001, 0.00001]
#         tau_dust = 1.76e-21 * (1500./1500.)**1.1 * s['SubhaloStarMetallicity'] * (0.11/0.44) * 5e21 #*min(1, max(0, 0.5*(8-zs[isnap])))
#         s['M1500_dust'] = s['M1500'] - 2.5*np.log10(np.exp(-tau_dust))
#         s['LyaPos'] = length_to_kpc * f['Subhalo']['LyaPos'][:][mask,:].astype(np.float64) # kpc
#         s['LyaVel'] = f['Subhalo']['LyaVel'][:][mask,:].astype(np.float64) # km/s
        s['LyaVelDisp'] = f['Subhalo']['LyaVelDisp'][:].astype(np.float64) # km/s
    
    # remove low LyaLum values
    otherfields = ['LyaCol', 'LyaRec', 'LyaStars', 'L1216', 'L1500', 'L2500', 'M1216', 'M1500', 'M2500', 'LyaVelDisp']
    mask = (s['Lya'] > 10**41.5)
    s['Lya'] = s['Lya'][mask]
    for field in fields:
        s[field] = s[field][mask]
    for field in otherfields:
        s[field] = s[field][mask]

    # remove high LyaLum values
    mask = (s['Lya'] < 10**45)
    s['Lya'] = s['Lya'][mask]
    for field in fields:
        s[field] = s[field][mask]
    for field in otherfields:
        s[field] = s[field][mask]
    
    return s

# s10 = read_Lya(snap=27) # z = 10
# s9 = read_Lya(snap=34) # z = 9
# s8 = read_Lya(snap=43) # z = 8
# s7 = read_Lya(snap=54) # z = 7
# s6 = read_Lya(snap=70) # z = 6
# s5 = read_Lya(snap=80) # z = 5.5

In [3]:
def read_spectra(snap, sim): #
    if sim=='Thesan-WC-2' or sim=='Thesan-sDAO-2':
        tau_dir = f'/nfs/mvogelsblab001/Lab/thesan-lya/{sim}/tau'
    elif sim=='Thesan-1':
        tau_dir = '/nfs/mvogelsblab001/Thesan/Thesan-1/postprocessing/tau'
    else:
        tau_dir=f'/pool001/users/claraxu/Thesan/{sim}/tau'
    s = {'snap': snap}
    filename = tau_dir + f'/full_stats/tau_{snap:03d}.hdf5'
    
    with h5py.File(filename, 'r') as f:
#         print([key for key in f.keys()])
#         print([item for item in f.attrs.items()])
        Dv_min = f.attrs['Dv_min']
        Dv_max = f.attrs['Dv_max']
        n_freq = f.attrs['NumFreq']
        s['n_bands'] = f.attrs['NumBands']
        s['freq'] = np.linspace(Dv_min, Dv_max, n_freq)
        
        for key in ['T_IGM_avg', 'TauIGM_16', 'TauIGM_50', 'TauIGM_84', 'TauIGM_avg']: # {freqs}
            s[key] = f[key][:]
        s['T_IGM_50'] = np.exp(-s['TauIGM_50'])
        s['T_IGM_16'] = np.exp(-s['TauIGM_16'])
        s['T_IGM_84'] = np.exp(-s['TauIGM_84'])
    
    return s

In [4]:
def read_transmission(snap, sim): # change this if you want to change the sightline
    if sim=='Thesan-WC-2' or sim=='Thesan-sDAO-2':
        tau_dir = f'/nfs/mvogelsblab001/Lab/thesan-lya/{sim}/tau'
    elif sim=='Thesan-1':
        tau_dir = '/nfs/mvogelsblab001/Thesan/Thesan-1/postprocessing/tau'
    else:
        tau_dir=f'/pool001/users/claraxu/Thesan/{sim}/tau'
        
    s_galaxies = {'snap': snap} # gotta stop calling everything s
    
    filename = tau_dir + f'/tau/tau_{snap:03d}.hdf5'
    
    with h5py.File(filename, 'r') as f:
#         print([key for key in f.keys()])
#         print([item for item in f.attrs.items()])
        Dv_min = f.attrs['Dv_min']
        Dv_max = f.attrs['Dv_max']
        n_freq = f.attrs['NumFreq']
        s_galaxies['freq'] = np.linspace(Dv_min, Dv_max, n_freq)
        
        for key in ['GroupIDs', 'SubhaloIDs']: 
            s_galaxies[key] = f[key][:] 
        s_galaxies['TauIGMs'] = f['TauIGMs'][:,0,:].astype(np.float64) # {galaxies, sightlines, freqs}. 0th sightline
        s_galaxies['T_IGM'] = np.exp(-s_galaxies['TauIGMs'])
    
    return s_galaxies

In [5]:
# model from Gangolli 2021: gaussian on the redward side, zero on blue side. M6
#       mu (i think) = beta * v_circ. beta is literally just tuned to fit observations. uh what do i do with that. seems a bit sus. gonna just use 1.5
#       FWHM = v_circ  => sigma = v_circ/2.355
#       v_circ is circular velocity of halo at boundary. what field is this in thesan?

# model from Weinberger 2019: gaussian offset redwards, no blue peak. M7
#       sigma = 88 km/s "as in Choudhury et al (2015)"
#       mu = v_circ * beta again. beta being either 1.5 or 1.8 depending on their reionization model. again tuned to fit observations
#       they also find that having a fixed mu does not fit observations

# model from Jensen 2013: mentions neufeld equation (i think) but then uses gaussian-minus-a-gaussian. M8
#       mu is same for both gaussians. but can't find where it is... is it 0? confused
#       sigma_1 = -6.5 + .75*log(halomass)
#       sigma_2 = -3.2 + .35*log(halomass)

# model in Byrohl and Gronke 2020: three
#       Neufeld, T = 1e4 K, N = 1e20 cm-2, kinda confused about how to use those parameters though. M9
#       Neufeld^ but only red peak. M10
#       gaussian at line center with width 200 km/s. hey it's my M1

def make_x_files(snap, sim, model):
    s_lya = read_Lya(snap, sim)
    s = read_spectra(snap, sim)
    s_galaxies = read_transmission(snap, sim)

    v = s['freq']
    veldisps = s_lya['LyaVelDisp'] 
    v_c = s_lya['SubhaloVelDisp']
    halomasses = s_lya['SubhaloMass']

    T_IGM_avg = s['T_IGM_50'] # an array over frequency
    T_IGM_galaxies = s_galaxies['T_IGM'] # a 2d array: galaxy, frequency at sightline 0
    n_centrals = len(T_IGM_galaxies)

    LyaLum = s_lya['Lya']

    n_galaxies = len(LyaLum)
    print(n_galaxies)

    Xs = np.zeros(n_galaxies) 

    transmission_ids = s_galaxies['SubhaloIDs']

    group_ids_centrals = s_galaxies['GroupIDs']
    group_ids_all = s_lya['SubhaloGrNr']

    T_IGM_groups = {} # dictionary of the transmission value of each group, keys are the group ids

    for i in range(n_centrals): # making the dict of centrals, transmission values for each group
        group_id = group_ids_centrals[i]
        T_IGM_groups[group_id] = T_IGM_galaxies[i]

    # this takes a really long time
    for i in range(n_galaxies): # calculating observed lum of each galaxy
#     for i in [1]:
        if model == 'M1':
            sigma = 200.
            mu = 0.
        elif model == 'M2':
            sigma = veldisps[i]
            if sigma < 88: 
                sigma = 88
            mu = 0.
        elif model == 'M3':
            sigma = veldisps[i]
            if sigma < 88: 
                sigma = 88
            mu = veldisps[i]
        elif model == 'M4':
            sigma = veldisps[i]
            if sigma < 88: 
                sigma = 88
            mu = v_c[i]
        elif model == 'M5':
            sigma = v_c[i]
            if sigma < 88: 
                sigma = 88
            mu = v_c[i]
        elif model == 'M6':
            sigma = v_c[i]/2.335
            if sigma < 88: 
                sigma = 88
            mu = 1.5 * v_c[i]
        elif model == 'M7':
            sigma = 88
            mu = 1.5 * v_c[i]
        elif model == 'M8':
            halomass = halomasses[i]
            sigma = -6.5 + .75*np.log10(halomass)
            sigma2 = -3.2 + .35*np.log10(halomass)
            mu = 0
        elif model == 'M9' or model == 'M10':
            a = 4.702e-4
            tau_0 = 5.9e6

        if model == 'M8':
            dlam = 1215.67 * v*km/c
            gaussian = np.exp(-0.5*((dlam)/sigma)**2) / sigma
            gaussian2 = np.exp(-0.5*((dlam)/sigma2)**2) / sigma2
            spectrum = gaussian - (sigma2/sigma) * gaussian2
        elif model != 'M9' and model != 'M10':
            gaussian = np.exp(-0.5*((v-mu)/sigma)**2) / sigma
            spectrum = gaussian
        elif model == 'M9':
            x = v/12.85
            at = a*tau_0
            neufeld = 0.5*np.pi * (np.sqrt(np.pi/6.)) * x**2 / (at * (np.cosh(np.sqrt((np.pi**3)/54.) * (x**3/at)))**2)
            spectrum = neufeld
        elif model == 'M10':
            x = v/12.85
            at = a*tau_0
            neufeld = 0.5*np.pi * (np.sqrt(np.pi/6.)) * x**2 / (at * (np.cosh(np.sqrt((np.pi**3)/54.) * (x**3/at)))**2)
            for j in range(400):
                neufeld[j] = 0
            spectrum = neufeld

        group_id = group_ids_all[i]

        if group_id in T_IGM_groups: # if this group has a centrals. aka if this group id is a key in the dict of keys of the centrals. 
            T_IGM = T_IGM_groups[group_id]
        else:
            T_IGM = T_IGM_avg

        idk = spectrum * T_IGM # thing to integrate. not sure what to call it
        numerator = np.sum(idk) 
        denominator = np.sum(spectrum)
        Xs[i] = numerator/denominator # problem: some are NaNs

#     write X file
    if sim=='Thesan-WC-2' or sim=='Thesan-sDAO-2':
        tau_dir = f'/nfs/mvogelsblab001/Lab/thesan-lya/{sim}/tau'
    elif sim=='Thesan-1':
        tau_dir = '/nfs/mvogelsblab001/Thesan/Thesan-1/postprocessing/tau'
    else:
        tau_dir=f'/pool001/users/claraxu/Thesan/{sim}/tau'  

    print(len(Xs))
    Xs = Xs[~np.isnan(Xs)]
    print(len(Xs))
    
    with h5py.File(f'{tau_dir}/X/X_{model}_0_{snap:03d}.hdf5', 'w') as f: 
        f.create_dataset('X', data=Xs, dtype=np.float32)


In [6]:
make_x_files(snap=60, sim='Thesan-1', model='M1')

Reading snapshot 60 ...
test0
test


<ipython-input-1-620816f089ae>:68: RuntimeWarning: divide by zero encountered in log10
  s['M1216'] = -2.5 * np.log10(fnu_1216_fac * s['L1216']) - 48.6 # Continuum absolute magnitude
<ipython-input-1-620816f089ae>:69: RuntimeWarning: divide by zero encountered in log10
  s['M1500'] = -2.5 * np.log10(fnu_1500_fac * s['L1500']) - 48.6 # Continuum absolute magnitude
<ipython-input-1-620816f089ae>:70: RuntimeWarning: divide by zero encountered in log10
  s['M2500'] = -2.5 * np.log10(fnu_2500_fac * s['L2500']) - 48.6 # Continuum absolute magnitude


13209
13209
13209
